In [ ]:
import sys
!{sys.executable} -m pip install storysniffer
!{sys.executable} -m pip install newspaper3k
!{sys.executable} -m pip install -U dill
!{sys.executable} -m pip install scikit-learn==1.0.2

In [ ]:
import requests
from urllib.parse import urlparse, urljoin
import sys
from bs4 import BeautifulSoup
import re
from datetime import datetime
import json
import urllib.parse
import hashlib
from storysniffer import StorySniffer
import ast
from newspaper import Article
import nltk
import pandas as pd

def is_valid(url):
    """
    Checks whether `url` is a valid URL.
    """
    parsed = urlparse(url)
    return bool(parsed.netloc) and bool(parsed.scheme)


def get_news_urls(url):
    # all URLs of `url`
    domain_name = urlparse(url).netloc
    urls = set()
    soup = BeautifulSoup(requests.get(url).content, "html.parser")
    for a_tag in soup.findAll("a"):
        href = a_tag.attrs.get("href")
        if href == "" or href is None:
            # href empty tag
            continue
        # join the URL if it's relative (not absolute link)
        href = urljoin(url, href)
        parsed_href = urlparse(href)
        # remove URL GET parameters, URL fragments, etc.
        href = parsed_href.scheme + "://" + parsed_href.netloc + parsed_href.path
        if not is_valid(href):
            # not a valid URL
            continue
        if href in internal_urls:
            # already in the set
            continue
        if domain_name not in href:
            # external link
            if href not in external_urls:
                external_urls.add(href)
            continue
        urls.add(href)
        internal_urls.add(href)

def crawl(news_url):
    print('get link ==> ', news_url)
    resp = requests.get(news_url)
    return resp.text

def get_host(url: str):
    token = url.split('://')[1]
    token = token.split('.')
    if token[0] == 'www':
        return token[1]
    return token[0]

In [ ]:
internal_urls = set()
external_urls = set()
    
url = 'http://www.riverfronttimes.com/'

urls = get_news_urls(url)

valid_urls = []
    
sniffer = StorySniffer()

for u in internal_urls:
    try:
        if ".pdf" in u: 
            continue
        if "/event" in u: 
            continue
        if "/places" in u: 
            continue
        if "/classifieds" in u:
            continue
        if "/special_sections" in u:
            continue
        if "/form" in u:
            continue
        if "/edition" in u:
            continue
        if "/archive" in u:
            continue
        if "/feed" in u:
            continue
        if "gallery" in u:
            continue
        if "/ads" in u:
            continue
        if "/promo" in u:
            continue
        if "/videos" in u:
            continue
        if "/members-only" in u:
            continue
#Video only series from the STL Post-Dispatch
        if "/the-bottom-line" in u:
            continue
#Removing the landing pages from the St. Louis Review
        if not u[-4:].isnumeric():
            continue
        if sniffer.guess(u) == True:
            valid_urls.append(u)

    except BaseException as e:
        pass
    except SSLError:
        pass
    except MaxRetryError:
        pass
    except SSLCertVerificationError:
        pass

In [ ]:
import sklearn

print('The scikit-learn version is {}.'.format(sklearn.__version__))

In [ ]:
internal_urls

In [ ]:
valid_urls

In [ ]:
news_data = []
# print(valid_urls)
try:
    for u in valid_urls:
        hash = hashlib.sha256(u.encode())
        news_data.append({
            'id': hash.hexdigest(),
            'url': u,
            'crawl_date': str(datetime.now()),
            'text': crawl(u),
            'host': urllib.parse.urlsplit(u).hostname
            })
except BaseException as e:
    pass

In [ ]:
news_data

In [ ]:
raw_articles = news_data

len(raw_articles)

In [ ]:
article = raw_articles[0]


In [ ]:
def given_tags(item,data):
    if data['hostname'] == 'missouri.statenews.net' and 'www.kansascity.com':
        tags = []
        soup = BeautifulSoup(article['text'],'html.parser')
        key = soup.find(attrs={"name":"keywords"})
        keyword = (key.get("content", None))
        tags.append(keyword) 
        return tags
    
    elif data['hostname'] == 'www.kansascity.com' or 'www.kmbc.com':
        tags = data['meta_data']['keywords']
        return tags
    
    else: 
        pass

def given_location(item,data):
    if data['hostname'] == 'fox2now.com':
        loc = []
        soup = BeautifulSoup(article['text'],'html.parser')
        key = soup.find(attrs={"name":"primaryCategory"})
        # keyword = (key.get("content", None))
        loc.append(key)
        return loc

    
def metadata(item,data):
    # metadata = data['meta_data']['keywords']
    
    return metadata
    

def given_date(item,data):
    
    if data['hostname'] == 'www.riverfronttimes.com':
        date = []
        soup = BeautifulSoup(article['text'],'html.parser')
        try:
            txt = soup.find("meta", attrs={"name":"date"})
            date.append(txt["content"])
            return date[0]
        
        except AttributeError:
            return ''
        
        except TypeError:
            return ''
    
    if data['hostname'] == 'www.archstl.org':
        date = []
        soup = BeautifulSoup(article['text'],'html.parser')
        try:
            txt = soup.find("div", {"class":"col-12 col-lg-3 ml-auto publish-date-meta text-right"}).get_text()
            txt_cleaned = txt.split('|', 1)[0].replace('Submitted','').replace('\r','').replace('\n','').strip()
            txt_2 = datetime.strptime(txt_cleaned, '%B %d, %Y')
            date.append(str(txt_2))
            return date[0]
        
        except AttributeError:
            return ''
        
        except TypeError:
            return ''
        
        except ValueError:
            return ''
    
    if data['hostname'] == 'callnewspapers.com':
        date = []
        soup = BeautifulSoup(article['text'],'html.parser')
        try:
            txt = soup.find("span", {"class":"time-wrapper"}).get_text().strip('\n')
            txt_cleaned = datetime.strptime(txt, '%B %d, %Y')
            date.append(str(txt_cleaned))
            return date[0]
        
        except AttributeError:
            return ''
        
        except TypeError:
            return ''
        
        except ValueError:
            return ''

    if data['hostname'] in ['www.timesnewspapers.com','www.bransontrilakesnews.com', 'stlamerican.com',
                           'www.stltoday.com']:
        date = []
        soup = BeautifulSoup(article['text'],'html.parser')
        try:
            txt = soup.find("meta", attrs={"itemprop":"dateCreated"})
            txt_cleaned = pd.to_datetime(txt["content"]).tz_convert('utc').tz_localize(None) 
            date.append(str(txt_cleaned))
            return date[0]
        
        except AttributeError:
            return ''
        
        except TypeError:
            return ''
     
    if data['hostname'] == 'sgfneighborhoodnews.com':
        date = []
        soup = BeautifulSoup(article['text'],'html.parser')
        try:
            txt = soup.find("span", {"class":"dtreviewed"}).get_text().strip('\n')
            txt_cleaned = datetime.strptime(txt, '%B %d, %Y')
            date.append(str(txt_cleaned))
            return date[0]
        
        except AttributeError:
            return ''
        
        except TypeError:
            return ''
        
        except ValueError:
            return ''
    
    if data['hostname'] == 'thesalemnewsonline.com':
        date = []
        soup = BeautifulSoup(article['text'],'html.parser')
        try:
            txt = soup.find("meta", attrs={"itemprop":"dateCreated"})
            txt_cleaned = pd.to_datetime(txt["content"]).tz_convert('utc').tz_localize(None) 
            date.append(str(txt_cleaned))
            return date[0]
        
        except AttributeError:
            return ''
        
        except TypeError:
            return ''
   
    if data['hostname'] == 'www.columbiamissourian.com':
        date = []
        soup = BeautifulSoup(article['text'],'html.parser')
        try:
            txt = soup.find("meta", attrs={"name":"parsely-pub-date"})
            txt_cleaned = pd.to_datetime(txt["content"]).tz_convert('utc').tz_localize(None) 
            date.append(str(txt_cleaned))
            return date[0]
        
        except AttributeError:
            return ''
        
        except TypeError:
            return ''
    
    if data['hostname'] == 'missouri.statenews.net':
        date = []
        soup = BeautifulSoup(article['text'],'html.parser') 
        key = soup.find(attrs={'class':'title_text'})
        temp = key.p.get_text(strip=True, separator='\n').splitlines()
        date.append(temp[1])
        return date
    
    elif data['hostname'] == 'www.kansascity.com':
        date = []
        soup = BeautifulSoup(article['text'],'html.parser')
        div = soup.find('div', attrs={'class':'publish-date'})
        srce = div.contents[0]
        date.append(srce)
        return date
    
    if data['hostname'] == 'www.gasconadecountyrepublican.com':
        date = data['meta_data']['twt-published-at']
        return date
    
    if data['hostname'] == 'www.hannibal.net':
        date = []
        soup = BeautifulSoup(article['text'],'html.parser')
        key  = soup.find('meta', attrs={'name': 'tncms-access-version'})
        content = key['content']
        date.append(content)
        return date
    
    else:
        
        date = data['date']
        return date
    

def source(item,data):
    if data['hostname'] == 'missouri.statenews.net': 
        src = []
        soup = BeautifulSoup(article['text'],'html.parser') 
        key = soup.find(attrs={'class':'title_text'})
        temp = key.p.get_text(strip=True, separator='\n').splitlines()
        src.append(temp[0])
        return src
    
    if data['hostname'] == 'abc17news.com':
        src = []
        soup = BeautifulSoup(article['text'],'html.parser')
        div = soup.find('div', attrs={'class':'meta__category'})
        srce = div.find('a').contents[0]
        src.append(srce)
        return src
    
    else: 
        
        pass


def author(item,data):
    
    if data['hostname'] == 'www.archstl.org':
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        #selector = 'div.col-8 > h3'
        try:
            txt = soup.find("div", attrs={"class":"author-meta d-flex flex-wrap justify-content-center flex-lg-column justify-content-lg-start"})
            auth.append(txt.get_text().replace('\n','').replace('\r','').strip())
            return auth[0]

        except AttributeError:
            return ''
        
        except TypeError:
            return ''
    
    if data['hostname'] == 'www.news-leader.com':
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            txt = soup.find("meta", attrs={"property":"article:author"})
            auth.append(txt["content"])
            return auth[0]

        except AttributeError:
            return ''
        
        except TypeError:
            return ''
    
    if data['hostname'] == 'www.semissourian.com':
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            txt = soup.find('div', {'class':['byline']}).get_text()
            auth.append(txt["content"])
            return auth[0]

        except AttributeError:
            return ''
        
        except TypeError:
            return ''
        
        try:
            txt = soup.find('a', {'class':['columnistbyline']}).get_text()
            auth.append(txt["content"])
            return auth[0]

        except AttributeError:
            return ''
        
        except TypeError:
            return ''
    
    if data['hostname'] in ['www.timesnewspapers.com','www.bransontrilakesnews.com']:
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            txt = soup.find("span", {"class":"reviewer"}).get_text().strip()
            auth.append(txt)
            return auth[0]

        except AttributeError:
            return ''
        
        except TypeError:
            return ''
    
    if data['hostname'] == 'sgfneighborhoodnews.com':
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            txt = soup.find("span", {"class":"reviewer"}).get_text().strip()
            auth.append(txt)
            return auth[0]

        except AttributeError:
            return ''
        
        except TypeError:
            return ''
    
    if data['hostname'] in ['thesalemnewsonline.com', 'stlamerican.com', 'www.pikecountynews.com', 
                            'newspressnow.com', 'callnewspapers.com', 'www.stltoday.com', 'www.riverfronttimes.com']:
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            txt = soup.find("meta", attrs={"name":"author"})
            auth.append(txt["content"].strip())
            return auth[0]

        except AttributeError:
            return ''
        
        except TypeError:
            return ''
    
    if data['hostname'] == 'lstribune.net':
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            txt =  soup.find("strong").get_text()
            if txt.startswith('By'):
                auth.append(txt)
                return auth[0]

        except AttributeError:
            return ''
        
        except TypeError:
            return ''
    
    
    if data['hostname'] == 'www.myleaderpaper.com':
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            txt = soup.find("span", {"class":"tnt-byline"}).get_text().strip()
            auth.append(txt)
            return auth[0]

        except AttributeError:
            return ''
        
        except TypeError:
            return ''
    
    if data['hostname'] == 'www.newstribune.com':
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            txt = soup.find("meta", attrs={"itemprop":"author"})
            auth.append(txt["content"])
            return auth[0]    

        except AttributeError:
            return ''
        
        except TypeError:
            return ''
    
    if data['hostname'] == 'www.greenecountycommonwealth.com':
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            txt = soup.find('div', class_='field-item even').get_text()
            if len(txt) <50:
                auth.append(txt)
                return auth[0]
            else: 
                return ''
            

        except AttributeError:
            return ''
        
        except TypeError:
            return ''
    
    if data['hostname'] == 'www.dddnews.com':
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            txt = soup.find('div', {'class':'byline'}).get_text()
            auth.append(txt)
            return auth[0]

        except AttributeError:
            return ''
        
        except TypeError:
            return ''
    
    if data['hostname'] == 'dailyjournalonline.com':
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            txt = soup.find("span", {"itemprop":"author"}).get_text().strip()
            auth.append(txt)
            return auth[0]

        except AttributeError:
            return ''
        
        except TypeError:
            return ''
    
    
    if data['hostname'] == 'www.darnews.com':
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            txt = soup.find('div', {'class':'byline'}).get_text()
            auth.append(txt)
            return auth[0]

        except AttributeError:
            return ''
        
        except TypeError:
            return ''

    if data['hostname'] == 'www.columbiamissourian.com':
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            txt = soup.find("meta", attrs={"name":"author"})
            auth.append(txt["content"])
            return auth[0]

        except AttributeError:
            return ''
        
        except TypeError:
            return ''
    
    if data['hostname'] == 'www.columbiatribune.com':
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            txt = soup.find("meta", property="article:author")
            auth.append(txt["content"])
            return auth[0]

        except AttributeError:
            return ''
        
        except TypeError:
            return ''
       
    
    if data['hostname'] == 'mycameronnews.com':
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            txt = soup.find('div', {'class':'field field-name-field-dateline field-type-text field-label-inline clearfix'}).get_text().strip('By:\xa0')
            auth.append(txt)
            return auth[0]

        except AttributeError:
            return ''
    
    if data['hostname'] == 'www.cassville-democrat.com':
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            txt = soup.find('div', {'class':['byline']}).get_text()
            auth.append(txt)
            return auth[0]

        except AttributeError:
            return ''
    
    if data['hostname'] == 'carthagenewsonline.com':
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            txt = soup.find("div", {"class": "td-post-author-name"})
            anchor = txt.find('a').get_text()
            auth.append(anchor)
            return auth[0]

        except AttributeError:
            return ''

    if data['hostname'] == 'www.californiademocrat.com':
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            txt = soup.find('a', attrs={'class':'bi-line-link'}).get_text()
            auth.append(txt)
            return auth[0]

        except AttributeError:
            return ''
    
    if data['hostname'] == 'auroraadvertiser.net':
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        spans = soup.find('span', attrs={'class':'byline'})
        auth.append(spans.text)

    
    if data['hostname'] == 'www.bransontrilakesnews.com':
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            txt = soup.find("span", { "itemprop" : "author" }).get_text()
            auth.append(txt)
            return auth[0]

        except AttributeError:
            return auth
    
    if data['hostname'] == 'www.memphisdemocrat.com':
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        txt_byline = soup.find(attrs={'class':'p2'})
        txt = soup.find("span", itemprop="name").text
        try:
            cleaned = txt_byline.get_text()
            if cleaned.startswith('By'): 
                auth.append(cleaned) 
                return auth[0]
           
        except AttributeError:
            return ''
        
        except TypeError:
            return ''
    
    
    if data['hostname'] == 'www.ksdk.com':
        auth = []
        soup = BeautifulSoup(article['text'], "html.parser")
        txt = soup.select_one('span:-soup-contains("Author:")').find_next_sibling(text=True)
        aut = txt.strip()
        auth.append(aut)
        return auth
    
    if data['hostname'] == 'www.ksmu.org':
        auth = []
        soup = BeautifulSoup(article['text'],'html.parser')
        div = soup.find('span', attrs={'class':'ArticlePage-authorBy'})
        srce = div.contents[1]
        # src.append(srce)
        aut = srce.contents[0]
        auth.append(aut)
        return auth
    
    if data['hostname'] == 'dosmundos.com':
        auth = data['meta_data']['twitter']['data1']
        return auth
    
    if data['hostname'] in ['houstonherald.com', 'stljewishlight.org']:
        auth= []
        soup = BeautifulSoup(article['text'],'html.parser')
        try:
            txt = soup.find("meta", attrs={"name":"twitter:data1"})
            auth.append(txt["content"].strip())
            return auth[0]
           
        except AttributeError:
            return ''
        
        except TypeError:
            return ''

    
    if data['hostname'] == 'www.gasconadecountyrepublican.com':
        auth = []
        aut = data['meta_data']['twt-author-name']
        word_list = aut
        auth = word_list[3:]
        return auth
    
    if data['hostname'] == 'www.hannibal.net':
        auth = data['meta_data']['author']
        return auth

    else:
        
        author = data['author']
        return author
    
def news(item,data):  
    
    if data['hostname'] == 'www.semissourian.com':
        nws = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            results=soup.find_all('p')
            nws.append(results.get_text())
            return nws[0]
            
        except AttributeError:
            return '' 

    if data['hostname'] == 'www.clintoncountyleader.com':
        nws = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            results=soup.find_all("div", {"class" : "paragraph"})
            for res in results:
                nws.append(res.text)
                return nws[0]
            
        except AttributeError:
            return '' 

    if data['hostname'] == 'stlamerican.com':
        nws = []
        soup = BeautifulSoup(article['text'], "html.parser")
        sep = 'TNCMS.AdManager.render'
        try:
            results=soup.find_all("div", {"itemprop" : "articleBody"})
            for res in results:
                nws.append(res.text.split(sep, 1)[0])
                return nws[0]
        
        except AttributeError:
            return ''  
    

    if data['hostname'] in ['www.timesnewspapers.com','www.bransontrilakesnews.com', 'www.columbiamissourian.com', 
                           'dailyjournalonline.com']:
        nws = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            results=soup.find_all("div", {"itemprop" : "articleBody"})
            for res in results:
                nws.append(res.text)
                return nws[0]
        
        except AttributeError:
            return ''    
    
    
    if data['hostname'] == 'standard-democrat.com':
        nws = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            results = soup.find('div',attrs={"class":"text"})
            nws.append(results.get_text())
            return nws[0]
        
        except AttributeError:
            return ''
        
        try:
            results_paywall = soup.find('div',attrs={"class":"pwblock"})
            nws.append(results.get_text())
            return nws[0]
        
        except AttributeError:
            return ''

    if data['hostname'] == 'thesalemnewsonline.com':
        nws = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            results=soup.find_all("div", {"itemprop" : "articleBody"})
            for res in results:
                nws.append(res.text)
                return nws[0]
        
        except AttributeError:
            return ''
    
    
    if data['hostname'] == 'www.pikecountynews.com':
        nws = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            results=soup.find_all("div", {"itemprop" : "articleBody"})
            for res in results:
                nws.append(res.text)
                return nws[0]
        
        except AttributeError:
            return ''
    
    if data['hostname'] == 'newspressnow.com':
        nws = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            results=soup.find_all("div", {"itemprop" : "articleBody"})
            for res in results:
                nws.append(res.text)
                return nws[0]
        
        except AttributeError:
            return ''
        
    if data['hostname'] == 'www.clintoncountyleader.com':
        nws = []
        soup = BeautifulSoup(article['text'], "html.parser")
        try:
            txt = soup.find('div', {'class':'blog-content'}).get_text()
            nws.append(txt)
            return nws[0]
        
        except AttributeError:
            return ''
        
    else:
        news = data['news']
        return news


def metadescription(item,data):
    if data['hostname'] == 'dosmundos.com':
        desc = []
        soup = BeautifulSoup(article['text'], "html.parser")
        descrip = soup.find("meta", property="og:description")
        des = descrip["content"]
        desc.append(des)
        return desc
    

def source_1(item,data):
    if data['hostname'] == 'abc17news.com' and 'fox2now.com':
        src = []
        soup = BeautifulSoup(article['text'],'html.parser')
        div = soup.find('div', attrs={'class':'meta__user vcard author'})
        srce = div.find('a').contents[0]
        src.append(srce)
        return src


def process_article(item):
    article = Article(url=item["url"])
    article.download(input_html=item["text"])

    article.parse()
    article.nlp()
    url = urlparse(item["url"])
        # print(hashlib.sha256(item['url'].encode()).hexdigest)
    data = {
            "id": hashlib.sha256(item["url"].encode()).hexdigest(),
            "title": str(article.title),
            "url": str(item["url"]),
            "news": str(article.text),
            "image": str(article.top_image),
            "date": str(article.publish_date),
            "author": article.authors,                                   # author derived from article.authors
            "meta_data": article.meta_data,
            "hostname": url.hostname,
            "crawl_datetime": str(datetime.now()),
            "stats": {},
            "tags" : {},                                                   # tags given by the outlet/reporter
            "inferred_tags_set1" : str(article.keywords),                 # tags derived from article.keywords
            "inferred_tags_set2" : {},                                    # tags inferred by our custom method
        
        }
    
    if len(data["news"]) > 10 and len(data["title"]) > 0:
        data['stats']['num_char'] = len(data['news'])
        
        date = given_date(item,data)
        data['date'] = date
        # if date =='None':
        #     data['date'] = str(article.publish_date)
        
        tags = given_tags(item,data)
        data['tags'] = tags
        
        src = source(item,data)
        data['source'] = src
        
        src = source_1(item,data)
        data['source_1'] = src
        
        auth = author(item,data)
        data['author'] = auth
        
        nws = news(item,data)
        data['news'] = nws

    
    
    return data
    


In [ ]:
curated_articles = []
for article in raw_articles: 
    curated_article = process_article(article)
    curated_articles.append(curated_article)

In [ ]:
curated_articles